In [ ]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import datetime as dt
import hvplot.pandas
import holoviews as hv
import panel as pn

In [ ]:
load_dotenv()

In [ ]:
ra_key = os.getenv("RAPIDAPI_KEY")
print(len(ra_key))

In [ ]:
url = "https://alpha-vantage.p.rapidapi.com/query"

eur_querystring = {"from_symbol":"EUR","function":"FX_DAILY","to_symbol":"USD","outputsize":"full","datatype":"csv"}
jpy_querystring = {"from_symbol":"USD","function":"FX_DAILY","to_symbol":"JPY","outputsize":"full","datatype":"csv"}
gbp_querystring = {"from_symbol":"GBP","function":"FX_DAILY","to_symbol":"USD","outputsize":"full","datatype":"csv"}
headers = {
    'x-rapidapi-key': ra_key,
    'x-rapidapi-host': "alpha-vantage.p.rapidapi.com"
    }

eur_response = requests.request("GET", url, headers=headers, params=eur_querystring)
jpy_response = requests.request("GET", url, headers=headers, params=jpy_querystring)
gbp_response = requests.request("GET", url, headers=headers, params=gbp_querystring)


In [ ]:
print('euro')
print(eur_response.text)
print('yen')
print(jpy_response.text)
print('pound')
print(gbp_response.text)

In [ ]:
eur_data = eur_response.text
eur_df = pd.DataFrame([x.split(',') for x in eur_data.splitlines()])
eur_df.columns = eur_df.iloc[0]
eur_df = eur_df[1:]

jpy_data = jpy_response.text
jpy_df = pd.DataFrame([x.split(',') for x in jpy_data.splitlines()])
jpy_df.columns = jpy_df.iloc[0]
jpy_df = jpy_df[1:]

gbp_data = gbp_response.text
gbp_df = pd.DataFrame([x.split(',') for x in gbp_data.splitlines()])
gbp_df.columns = gbp_df.iloc[0]
gbp_df = gbp_df[1:]

In [ ]:
curr_fx = pd.DataFrame()
curr_fx = pd.concat([eur_df, jpy_df, gbp_df], axis=1, sort=False)
curr_fx.head()

In [ ]:
curr_fx.columns = ['datetime', 'eur_open', 'eur_high', 'eur_low', 'eur_close', 'jpy_date', 'jpy_open', 'jpy_high', 'jpy_low', 'jpy_close', 'gbp_date', 'gbp_open', 'gbp_high', 'gbp_low', 'gbp_close']
curr_fx = curr_fx.drop(columns=['jpy_date', 'gbp_date'])
curr_fx.head()

In [ ]:
curr_fx['datetime'] = pd.to_datetime(curr_fx['datetime'])
curr_fx.set_index('datetime', inplace=True)

In [ ]:
curr_fx.head()

In [ ]:
curr_fx['eur_open'] = curr_fx['eur_open'].astype(float)
curr_fx['eur_high'] = curr_fx['eur_high'].astype(float)
curr_fx['eur_low'] = curr_fx['eur_low'].astype(float)
curr_fx['eur_close'] = curr_fx['eur_close'].astype(float)
curr_fx['jpy_open'] = curr_fx['jpy_open'].astype(float)
curr_fx['jpy_high'] = curr_fx['jpy_high'].astype(float)
curr_fx['jpy_low'] = curr_fx['jpy_low'].astype(float)
curr_fx['jpy_close'] = curr_fx['jpy_close'].astype(float)
curr_fx['gbp_open'] = curr_fx['gbp_open'].astype(float)
curr_fx['gbp_high'] = curr_fx['gbp_high'].astype(float)
curr_fx['gbp_low'] = curr_fx['gbp_low'].astype(float)
curr_fx['gbp_close'] = curr_fx['gbp_close'].astype(float)
curr_fx.info()

In [17]:
curr_fx.hvplot(x='datetime', y=['eur_close',],width=800, height=600, grid=True)

:Curve   [datetime]   (eur_close)

In [ ]:
curr_fx.hvplot(x='datetime', y=['jpy_close',],width=800, height=600, grid=True)

In [ ]:
curr_fx.hvplot(x='datetime', y=['gbp_close'],width=800, height=600, grid=True)

In [16]:
y = pn.widgets.Select(name='currency', options=['eur_open',
                                                'eur_high',
                                                'eur_low',
                                                'eur_close',
                                                'jpy_open',
                                                'jpy_high',
                                                'jpy_low',
                                                'jpy_close',
                                                'gbp_open',
                                                'gbp_high',
                                                'gbp_low',
                                                'gbp_close'])

plot1 = curr_fx.hvplot(y=y, width=600, height=400, grid=True)


pn.Tabs(('currency1',pn.Column(plot1, y)))


Tabs
    [0] Column
        [0] ParamFunction(function)
        [1] Select(name='currency', options=['eur_open', 'eur_high', ...], value='eur_open')